In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers

# Классические LSTM

In [ ]:
lstm = tf.keras.Sequential()
# Добавим слой Embedding ожидая на входе словарь размера 1000, и
# на выходе вложение размерностью 64.
lstm.add(layers.Embedding(input_dim=1000, output_dim=64))

# Добавим слой LSTM с 128 внутренними узлами.
lstm.add(layers.LSTM(128))

# Добавим слой Dense с 10 узлами и активацией softmax.
lstm.add(layers.Dense(10))

lstm.summary()

# GRU сеть

In [ ]:
GRU = tf.keras.Sequential()
GRU.add(layers.Embedding(input_dim=1000, output_dim=64))

# Выходом GRU будет 3D тензор размера (batch_size, timesteps, 256)
GRU.add(layers.GRU(256, return_sequences=True))

# Выходом SimpleRNN будет 2D тензор размера (batch_size, 128)
GRU.add(layers.SimpleRNN(128))

GRU.add(layers.Dense(10))

GRU.summary()

In [ ]:
encoder_vocab = 1000
decoder_vocab = 2000

encoder_input = layers.Input(shape=(None, ))
encoder_embedded = layers.Embedding(input_dim=encoder_vocab, output_dim=64)(encoder_input)

# Возвращает состояния в добавление к выходным данным
output, state_h, state_c = layers.LSTM(
    64, return_state=True, name='encoder')(encoder_embedded)
encoder_state = [state_h, state_c]

decoder_input = layers.Input(shape=(None, ))
decoder_embedded = layers.Embedding(input_dim=decoder_vocab, output_dim=64)(decoder_input)

# Передает 2 состояния в новый слой LSTM в качестве начального состояния
decoder_output = layers.LSTM(
    64, name='decoder')(decoder_embedded, initial_state=encoder_state)
output = layers.Dense(10)(decoder_output)

lstm_2 = tf.keras.Model([encoder_input, decoder_input], output)
lstm_2.summary()

# Кросс-пакетное сохранение состояния

In [ ]:
lstm_layer = layers.LSTM(64, stateful=True)
for s in sub_sequences:
    output = lstm_layer(s)

In [ ]:
paragraph1 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph2 = np.random.random((20, 10, 50)).astype(np.float32)
paragraph3 = np.random.random((20, 10, 50)).astype(np.float32)

lstm_layer = layers.LSTM(64, stateful=True)
output = lstm_layer(paragraph1)
output = lstm_layer(paragraph2)
output = lstm_layer(paragraph3)

# reset_states() сбосит кешированное состояние до изначального initial_state.
# Если initial_state не было задано, по умолчанию будут использованы нулевые состояния.
lstm_layer.reset_states()

# Двунаправленные RNN

In [ ]:
model = tf.keras.Sequential()

model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True), 
                               input_shape=(5, 10)))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(10))

model.summary()

# Оптмимзация производительности и ядра CuDNN 

In [ ]:
batch_size = 64
# Каждый пакет изображений MNIST это тензор размерностью (batch_size, 28, 28).
# Каждая входная последовательность размера (28, 28) (высота рассматривается как время).
input_dim = 28

units = 64
output_size = 10  # метки от 0 до 9

# Построим RNN модель
def build_model(allow_cudnn_kernel=True):
  # CuDNN доступен только на уровне слоя, а не на уровне ячейки.
  # Это значит `LSTM(units)` будет использовать ядро CuDNN,
  # тогда как RNN(LSTMCell(units)) будет использовать non-CuDNN ядро.
    if allow_cudnn_kernel:
        # Слой LSTM с параметрами по умолчанию использует CuDNN.
        lstm_layer = tf.keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        # Обертка LSTMCell слоем RNN не будет использовать CuDNN.
        lstm_layer = tf.keras.layers.RNN(
            tf.keras.layers.LSTMCell(units),
            input_shape=(None, input_dim))
        model = tf.keras.models.Sequential([
          lstm_layer,
          tf.keras.layers.BatchNormalization(),
          tf.keras.layers.Dense(output_size)]
        )
    return model

# Пример с MNIST

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
sample, sample_label = x_train[0], y_train[0]

In [ ]:
model = build_model(allow_cudnn_kernel=True)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=batch_size,
          epochs=5)

# Новая модель без CuDDN

In [ ]:
slow_model = build_model(allow_cudnn_kernel=False)
slow_model.set_weights(model.get_weights())
slow_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                   optimizer='sgd', 
                   metrics=['accuracy'])
slow_model.fit(x_train, y_train, 
               validation_data=(x_test, y_test), 
               batch_size=batch_size,
               epochs=1)  # Обучим только за одну эпоху потому что она медленная.